## How to run DETR for MrOS dataset step by step

To run the DETR model with MrOs data, we have modified the original [mros-data](https://github.com/neergaard/mros-data) SleepEventDataModule, and integrated it with the [DETR code](https://github.com/facebookresearch/detr) for training and testing. By cloning this repository, you already have access to the modified changes. Now we will describe the steps you have to follow to run the DETR model on MrOS data.

### 1. Download the data

First, you will need to download the preprocessed subject data as .h5 files in `deep-learning-project/data/preprocessed-sdb/` folder. Please check out the `README.md` in the `mros_data/preprocessing/` directory of the [mros-data](https://github.com/neergaard/mros-data) module for instructions on how to convert raw `EDF` to `H5` files. An example of a subject record in this format can be found in this folder in the repository.

### 2. Install the required packages

To install the packages, we recommend to do so in a project environment. If you are working on the DTU cluster, you can first [install mini conda](https://waylonwalker.com/install-miniconda/), create and enviornment for this project, and install the packages there. Make sure to activate your environment before starting the package installation. 

It is important to note that this model needs CUDA to work, therefore we recommend you to work on a server with such feature.

Also, the paths in this notebook are relative to the location of the current notebook.

#### 2.1 Install required packages for DETR

In [ ]:
! pip install ../detr_mros/requirements.txt

#### 2.2 Install required packages for mros-data 

In [ ]:
! pip install librosa mne torch pytorch-lightning rich ipympl pytest h5py

In [ ]:
# Move to mros-data folder and install 
%cd "../src/mros-data/"

In [ ]:
os.getcwd()

In [ ]:
! pip install -e .

### 3. Select data used by main.py for model training and evaluation

From the set of subject files stored in `preprocessed-sdb`, copy those you want to use for training and place them inside `deep-learning-project/detr-mros/train/`. These will be further splitted into train/evaluation sets according to the parameters defined in the `params` dictionary in main.py (line 151). Change the batch size and n_eval (these are the subjects used for evaluation out of your total records stored in /train) according to your preferences.

### 4. Select data used by test.py for testing 

From the set of subject files stored in `preprocessed-sdb`, copy those you want to use for predicting their boxes and place them inside `deep-learning-project/detr-mros/test/input`.

### 5. Train the model 

Since the model training may take hours, we recommend inserting the following command in a job file and scheduling it on a server. This is an example of training with specific hyperparameters. You can check the hyperparameters that can be selected in detr-mros/main.py. 

In [ ]:
# Change to detr-mros folder
%cd "../../detr-mros/"

In [ ]:
! python main.py --output_dir output --epochs 5000  --dropout 0.5 --enc_layers 3 --dec_layers 3 --lr 1e-6 --num_queries 10 --lr_drop 5000

### 6. Test the model 

Once you have trained the model and the weights have been calculated, these will be stored in `detr-mros/output/checkpoint.pth`. These can be loaded and used for predicting the boxes on any record. The output of this script will be stored in `detr-mros/test/output`. It consists of: 
1. results.txt: Which includes the scaled and unscaled predicted boxes for each segment.
2. Spectogram plot with predicted (green) and ground truth (red) boxes.
3. Signal plot with predicted (green) and ground truth (red) boxes.

An example of each of these output files can be found in `detr-mros/test/output` in the repository.

In [ ]:
! python test.py --resume output/checkpoint.pth --epochs 5000  --dropout 0.5 --enc_layers 3 --dec_layers 3 --lr 1e-6 --num_queries 10 --lr_drop 5000